In [ ]:
import respiration.dataset as repository

dataset = repository.from_default()

subject = 'Proband16'
scenario = '101_natural_lighting'

In [ ]:
frames, meta = dataset.read_video_bgr(subject, scenario)

In [ ]:
from respiration.extractor.mtts_can import preprocess_video_frames

resized, normalized = preprocess_video_frames(frames)

In [ ]:
import matplotlib.pyplot as plt

# Plot resized and normalized frames
_, axs = plt.subplots(1, 2, figsize=(15, 5))

axs[0].imshow(resized[0])
axs[0].set_title('Resized Frame')

axs[1].imshow(normalized[0])
axs[1].set_title('Normalized Frame')

In [ ]:
# Save the normalized_frames as a video
# import cv2
# 
# fourcc = cv2.VideoWriter_fourcc(*'XVID')
# out = cv2.VideoWriter('normalized_frames.avi', fourcc, 30.0, (36, 36))
# 
# for frame in normalized_frames:
#     frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
#     frame = (frame * 255).astype('uint8')  # Scale and convert to 8-bit
#     out.write(frame) 
#     
# out.release()

In [ ]:
import os
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt

from respiration.extractor.mtts_can import mtts_can

model_checkpoint = os.path.join('..', 'data', 'mtts_can', 'mtts_can.hdf5')

frame_depth = 10

# The model expects a number of frames that is a multiple of frame_depth
cut_off = (normalized.shape[0] // frame_depth) * frame_depth
input_resized = resized[:cut_off]
input_normalized = normalized[:cut_off]

cut_off, frame_depth

In [ ]:
input_resized.shape

In [ ]:
model = mtts_can(frame_depth, 32, 64, (36, 36, 3))
model.load_weights(model_checkpoint)
yptest = model.predict(
    (input_resized, input_normalized),
    batch_size=100
)

In [ ]:
import numpy as np
from respiration.preprocessing import detrend_tarvainen

pulse_pred = yptest[0]
pulse_pred = detrend_tarvainen(np.cumsum(pulse_pred), 100)
[b_pulse, a_pulse] = butter(1, [0.75 / meta.fps * 2, 2.5 / meta.fps * 2], btype='bandpass')
pulse_pred = filtfilt(b_pulse, a_pulse, np.double(pulse_pred))

resp_pred = yptest[1]
resp_pred = detrend_tarvainen(np.cumsum(resp_pred), 100)
[b_resp, a_resp] = butter(1, [0.08 / meta.fps * 2, 0.5 / meta.fps * 2], btype='bandpass')
resp_pred = filtfilt(b_resp, a_resp, np.double(resp_pred))

In [ ]:
import respiration.preprocessing as sp

fig, axs = plt.subplots(4, 1, figsize=(20, 20))

# Add some space between the plots
fig.subplots_adjust(hspace=0.5)

axs[0].plot(pulse_pred, label='Predicted Pulse')
axs[0].set_title('Predicted Pulse')
axs[0].set_xlabel('Time')
axs[0].set_ylabel('Pulse')
axs[0].legend()

axs[1].plot(resp_pred, label='Predicted Respiration')
axs[1].set_title('Predicted Respiration')
axs[1].set_xlabel('Time')
axs[1].set_ylabel('Respiration')
axs[1].legend()

resp_pred_2 = np.cumsum(yptest[1])
# resp_pred_2 = detrend(resp_pred_2, 100)
resp_pred_2 = sp.standard_processing(resp_pred_2, meta.fps, 0.08, 0.5)

axs[2].plot(resp_pred_2, label='Predicted Respiration')
axs[2].set_title('Predicted Respiration')
axs[2].set_xlabel('Time')
axs[2].set_ylabel('Respiration')
axs[2].legend()

gt_signal, gt_sample_rate = dataset.get_ground_truth_rr_signal(subject, scenario)
gt_signal = sp.standard_processing(gt_signal, gt_sample_rate, 0.08, 0.5)

axs[3].plot(gt_signal, label='Ground Truth Respiration')
axs[3].set_title('Ground Truth Respiration')
axs[3].set_xlabel('Time')
axs[3].set_ylabel('Respiration')
axs[3].legend()

In [ ]:
from respiration.analysis import FrequencyExtractor

gt_fe = FrequencyExtractor(gt_signal, gt_sample_rate, lowpass=0.1, highpass=0.5)
pd_fe = FrequencyExtractor(resp_pred_2, meta.fps, lowpass=0.1, highpass=0.5)

gt_bpm = gt_fe.frequency_from_nfcp() * 60
pd_bpm = pd_fe.frequency_from_nfcp() * 60

abs(gt_bpm - pd_bpm)